# Potrzebne importy oraz nasze skrypty

In [ ]:
import os
from dotenv import load_dotenv
import glob
load_dotenv()
from qdrant import QdrantDB
from vector import get_openai_embedding
import uuid


# Klasa służąca do komunikacji z bazą danych


```python

class QdrantDB:
    def __init__(self, url: str = "localhost", port: int = 6333, api_key: Optional[str] = None):
        """
        Initialize the Qdrant client.

        Args:
            url: URL of the Qdrant server
            port: Port of the Qdrant server
            api_key: API key for authentication (if needed)
        """
        if api_key:
            self.client = QdrantClient(url=url, port=port, api_key=api_key, check_compatibility=False)
        else:
            self.client = QdrantClient(url=url, port=port, check_compatibility=False)
```






In [ ]:
qdrant_client = QdrantDB(url="localhost", port=6333)

# Tworzenie kolekcji

In [ ]:
collection_name = "notatki"
vector_size = 3072  # Rozmiar modelu
success = qdrant_client.create_collection(
    collection_name=collection_name,
    vector_size=vector_size,
    distance="Cosine"
)
print(f"Collection created: {success}")

# Lista dostępnych kolekcji

In [ ]:
collections = qdrant_client.list_collections()
print("Available collections:")
for collection in collections:
    print(f"- {collection.name}")

# Dodawanie dokumentów do kolekcji

```python
    def create_collection(self, collection_name: str, vector_size: int, distance: str = "Cosine"):
        """
        Create a new collection in Qdrant.

        Args:
            collection_name: Name of the collection
            vector_size: Dimensionality of vectors
            distance: Distance metric ("Cosine", "Euclid", or "Dot")

        Returns:
            True if collection was created successfully
        """
        try:
            self.client.create_collection(
                collection_name=collection_name,
                vectors_config=models.VectorParams(
                    size=vector_size,
                    distance=distance
                )
            )
            return True
        except Exception as e:
            print(f"Error creating collection: {e}")
            return False
```

In [ ]:
notes_directory = "Documents/pages"

markdown_files = glob.glob(os.path.join(notes_directory, "*.md"))

print(f"Found {len(markdown_files)} markdown files")

for file_path in markdown_files:
    try:
        filename = os.path.basename(file_path)

        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()

        vector = get_openai_embedding(content)

        payload = {
            "filename": filename,
            "source_path": file_path
        }

        file_id = str(uuid.uuid4())

        success = qdrant_client.add_point(
            collection_name=collection_name,
            id=file_id,
            vector=vector,
            payload=payload
        )

        if success:
            print(f"Added file '{filename}' to collection with ID '{file_id}'")
        else:
            print(f"Failed to add file '{filename}'")

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

# Wyszukiwanie dokumentów w kolekcji

```python

vector = get_openai_embedding(args.text) # wazane bo zapytanie tez musi być w formie wektora
    def search(self, collection_name: str, query_vector: List[float], limit: int = 10,
               filter: Optional[models.Filter] = None):
        """
        Search for similar vectors in the collection.

        Args:
            collection_name: Name of the collection
            query_vector: Vector to search for
            limit: Maximum number of results
            filter: Optional filter conditions

        Returns:
            List of search results with points
        """
        try:
            return self.client.search(
                collection_name=collection_name,
                query_vector=query_vector,
                limit=limit,
                query_filter=filter
            )
        except Exception as e:
            print(f"Error searching: {e}")
            return []
```

In [27]:
query_text = "AI and machine learning technologies"
query_vector = get_openai_embedding(query_text)

search_results = qdrant_client.search(
    collection_name=collection_name,
    query_vector=query_vector,
    limit=3
)
print(f"\nSearch results for: '{query_text}'")
for result in search_results:
    print(f"ID: {result.id}, Score: {result.score}")
    if hasattr(result, 'payload') and result.payload:
        print(f"Text: {result.payload.get('filename', 'N/A')}")
    print("---")


Search results for: 'AI and machine learning technologies'
ID: f719f9ab-d51e-4598-a9db-dd7f06653d95, Score: 0.6133485
Text: Artificial Intelligence.md
---
ID: 99275c04-dc23-43ae-a497-153f7872a781, Score: 0.56131697
Text: Machine Learning.md
---
ID: aa4edb9c-4359-431a-9169-c9be003d61f3, Score: 0.54975057
Text: Generative Artificial Intelligence.md
---


# Informacje o konkretnym punkcie

In [28]:
point = qdrant_client.get_point(collection_name=collection_name, id="aa4edb9c-4359-431a-9169-c9be003d61f3")
print(f"\nRetrieved point by ID:")
if point and hasattr(point, 'payload'):
    print(f"Text: {point.payload.get('filename', 'N/A')}")



Retrieved point by ID:
Text: Generative Artificial Intelligence.md


# Aktualizacja punktu

Aktualizacja punktu również wymaga zmiany wektora [Qdrant API - Upsert Points](https://api.qdrant.tech/api-reference/points/upsert-points)


In [33]:
id = "aa4edb9c-4359-431a-9169-c9be003d61f3"
updated_payload = {
    "category": "artificial intelligence",
    "importance": "high"
}
success = qdrant_client.update_payload(
    collection_name=collection_name,
    id=id,
    payload=updated_payload
)

success = qdrant_client.add_point(
    collection_name=collection_name,
    id=id,
    vector=[0.1, 0.2, 0.3],  # Example vector, should be the same size as the original
    payload=updated_payload
)

print(f"Payload updated: {success}")

Error adding points: Unexpected Response: 400 (Bad Request)
Raw response content:
b'{"status":{"error":"Wrong input: Vector dimension error: expected dim: 3072, got 3"},"time":0.000760417}'
Payload updated: False


# Liczebnośc kolekcji

In [31]:
count = qdrant_client.count_points(collection_name=collection_name)
print(f"\nTotal points in collection: {count}")


Total points in collection: 67
